In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import seaborn as sns
import math
import datetime
import keras
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler
from sklearn.utils import class_weight
from keras import optimizers
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.models import load_model
import joblib
import statistics
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
path_var = 1

if (path_var == 1):
    filepath = '/home/docboy/Desktop/Option_Pricing/datasets/'
    modelpath = '/home/docboy/Desktop/Option_Pricing/pickles/'


else:
    filepath = '/home/sharan/Desktop/OptionPricing-master/datasets/'
    modelpath = '/home/sharan/Desktop/OptionPricing-master/pickles/'



In [3]:
def c_int(lst):
    df = pd.DataFrame(lst)
    df = df.sort_values(by = 0).reset_index(drop=True)
    cp1 = df[np.logical_and(df[0] >= -2,df[0] <= 2)]
    return (len(cp1)/len(df))*100

In [19]:
len(lower_p_bound)

2679

In [37]:
nfty = filepath + 'NIFTY50_feature_set_COV_complete.csv'
nfty_ecdf = filepath + 'NIFTY50_feature_set_ECDF_complete.csv'

nfty_19= filepath + 'NIFTY50_2019_feature_set_COV_complete.csv'
nfty_ecdf_19= filepath + 'NIFTY50_2019_feature_set_ECDF_complete.csv'

bnfty = filepath + 'BANKNIFTY_feature_set_COV_complete.csv'
bnfty_ecdf = filepath + 'BANKNIFTY_feature_set_ECDF_complete.csv'


for approach in range(3, 4):

    if (approach == 1):
        feats = pd.read_csv(nfty_ecdf)
        print ('\n\nApproach var set to ECDF')


        dropNames = ['date','percentage','strike','spot','opPrice']

    else:
        feats = pd.read_csv(nfty)
        print ('\n\nApproach var set to :: '+ str(approach))


        if (approach == 2):
            dropNames= ['date','percentage','strike','spot','opPrice','change_oi',
                            'ts_atr','prevOp','avg_atmError']

        else:
            dropNames= ['date', 'percentage','strike','spot','opPrice','ts_atr','change_oi']

        ###################
        
    if (approach == 1):
        dev = 100
    else:
        dev = 1

        
    feats['date'] = pd.to_datetime(feats['date'])
    feats = feats[(feats['date']>=datetime.datetime(2017, 10, 1))]
    feats = feats.reset_index(drop=True)
    
    
    X=feats.iloc[:,:-1]
    Y=feats.loc[:,'Target']
    X = X.drop(dropNames, axis = 1)    

    # Load the scalar file
    scaler = joblib.load(modelpath + 'Combined_DL_SCALER_NIFTY_0.02_Expt' + str(approach) + '.model')

    # Start the scaling process

    featNames = list(X)  
    X = scaler.transform(X)
    X = pd.DataFrame(X, columns = featNames)


    onehot = joblib.load(modelpath + 'Combined_DL_ONEHOT_NIFTY_0.02_Expt' + str(approach) + '.model')

    Y = Y.values
    Y = Y.reshape(len(Y), 1)
    Y = onehot.transform(Y)

    # Split the train and test sets without random optimization
    
    
    X_test = X.iloc[:, :]
    Y_test = Y[:] 
    print(len(Y_test))

    # Saving the model
    model = joblib.load(modelpath + 'Combined_DL_NIFTY_0.02_Expt' + str(approach) + '.model')

    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test accuracy:', score[1]*100)

    Y_pred=model.predict(X_test)
    Y_pred=np.argmax(Y_pred,axis=1)

    OGY_test=onehot.inverse_transform(Y_test)
    # OGY_test=sum(OGY_test, [])
    OGY_test=OGY_test.ravel()
    # print(OGY_test)

    bin_width = 0.1
    # Normalized Error Metric (NEM)

    dist_sum = 0
    for i in range(len(Y_pred)):
        dist = abs(Y_pred[i] - OGY_test[i])
        dist_sum = dist_sum + dist

    print("NEM:{}".format((dist_sum*bin_width)/len(Y_test)))

    CP=[]
    for i in range(len(Y_pred)):
        dist = OGY_test[i] - Y_pred[i]
        CP.append(dist)
    CP=np.array(CP,dtype=float)

    print("Confidence Interval:{}".format(c_int(CP)))

    dist_sum = 0
    step = 0.1

    date=[]
    lower_p_bound=[]
    higher_p_bound=[]
    actual_p=[]
        
    lower_int_p = []
    upper_int_p = []
        
        
################################

    for i in range(len(Y_pred)):    
        lower_bound_opPrice = ((Y_pred[i]*step) * (feats.loc[i,'strike']))/100
        lower_p_bound.append(lower_bound_opPrice)

################################

    for i in range(len(Y_pred)):    

        higher_bound_opPrice = (((Y_pred[i]+1) * step) * (feats.loc[i,'strike']))/100    
        higher_p_bound.append(higher_bound_opPrice)


################################
    for i in range(len(Y_pred)):    

        actual_opPrice = feats.loc[i,'opPrice']
        actual_p.append(actual_opPrice)                    

################################
    for i in range(len(Y_pred)):    

        lower_lim_opPrice = (((Y_pred[i]-2) * step) * (feats.loc[i,'strike']))/100    
        lower_int_p.append(lower_lim_opPrice)    

################################
    for i in range(len(Y_pred)):    

        upper_lim_opPrice = (((Y_pred[i]+3) * step) * (feats.loc[i,'strike']))/100    
        upper_int_p.append(upper_lim_opPrice)    
    
        
    priceval_dataSet=pd.DataFrame()
    priceval_dataSet['date']=feats['date']
    priceval_dataSet['lower_p']=lower_p_bound
    priceval_dataSet['higher_p']=higher_p_bound
    priceval_dataSet['actual_p']=actual_p
    priceval_dataSet['LowerLim_p'] = lower_int_p
    priceval_dataSet['UpperLim_p'] = upper_int_p
        
        
        
        # print(len(priceval_dataSet))
    priceval_dataSet= priceval_dataSet[(priceval_dataSet['lower_p']!=10000)]
    priceval_dataSet= priceval_dataSet[(priceval_dataSet['higher_p']!=10000)]
    priceval_dataSet= priceval_dataSet[(priceval_dataSet['actual_p']!=10000)]
    priceval_dataSet= priceval_dataSet[(priceval_dataSet['LowerLim_p']!=10000)]
    priceval_dataSet= priceval_dataSet[(priceval_dataSet['UpperLim_p']!=10000)]
        
    print(len(priceval_dataSet))
#     priceval_dataSet=priceval_dataSet.groupby('date').mean()
    decimals = 3    
    priceval_dataSet['higher_p'] = priceval_dataSet['higher_p'].apply(lambda x: round(x, decimals))
    priceval_dataSet['lower_p'] = priceval_dataSet['lower_p'].apply(lambda x: round(x, decimals))
    priceval_dataSet['actual_p'] = priceval_dataSet['actual_p'].apply(lambda x: round(x, decimals))
    priceval_dataSet['LowerLim_p'] = priceval_dataSet['LowerLim_p'].apply(lambda x: round(x, decimals))
    priceval_dataSet['UpperLim_p'] = priceval_dataSet['UpperLim_p'].apply(lambda x: round(x, decimals))
        
    print((priceval_dataSet))
        
    
    priceval_dataSet.to_csv(filepath+"price_vals/Combined_DL_NIFTY_approach_"+str(approach)+".csv", index = False)

#     priceval_dataSet['minA']=abs(priceval_dataSet['actual_p']-priceval_dataSet['higher_p'])
#     priceval_dataSet['minB']=abs(priceval_dataSet['actual_p']-priceval_dataSet['lower_p'])
#     priceval_dataSet['min']=priceval_dataSet[['minA','minB']].min(axis=1)

#     print("Mean={}".format(priceval_dataSet['min'].mean()))

#     print("Median={}".format(priceval_dataSet['min'].median()))



Approach var set to :: 3
2679
Test accuracy: 29.8618882894516
NEM:0.13699141470698023
Confidence Interval:83.42665173572227
2679
           date  lower_p  higher_p  actual_p  LowerLim_p  UpperLim_p
0    2017-10-03    232.8    242.50     221.2       213.4      261.90
1    2017-10-03    118.2    128.05     120.3        98.5      147.75
2    2017-10-03     10.2     20.40      12.2       -10.2       40.80
3    2017-10-03     20.3     30.45      17.6         0.0       50.75
4    2017-10-03     20.2     30.30      24.7         0.0       50.50
...         ...      ...       ...       ...         ...         ...
2674 2018-09-26    168.0    179.20     149.3       145.6      201.60
2675 2018-09-26     91.2    102.60      70.8        68.4      125.40
2676 2018-09-26     90.8    102.15      87.9        68.1      124.85
2677 2018-09-26    222.0    233.10     203.9       199.8      255.30
2678 2018-09-26    275.0    286.00     265.8       253.0      308.00

[2679 rows x 6 columns]


In [38]:
priceval_dataSet

,date,lower_p,higher_p,actual_p,LowerLim_p,UpperLim_p
0,2017-10-03,232.8,242.50,221.2,213.4,261.90
1,2017-10-03,118.2,128.05,120.3,98.5,147.75
2,2017-10-03,10.2,20.40,12.2,-10.2,40.80
3,2017-10-03,20.3,30.45,17.6,0.0,50.75
4,2017-10-03,20.2,30.30,24.7,0.0,50.50
...,...,...,...,...,...,...
2674,2018-09-26,168.0,179.20,149.3,145.6,201.60
2675,2018-09-26,91.2,102.60,70.8,68.4,125.40
2676,2018-09-26,90.8,102.15,87.9,68.1,124.85
2677,2018-09-26,222.0,233.10,203.9,199.8,255.30


In [39]:
df = priceval_dataSet

In [41]:
# df.plot()